## Two satisfactory(2-SAT)
- 충족 가능성 문제(Satisfiability problem)는 불리언 변수들의 집합에 대한 불리언 식이 참이 되는지를 결정하는 문제이다.
  - 여러개의 Boolean 변수들로 이루어진 식이 있을 때, 각 변수에 값을 할당하여 식이 참이되는 조합을 찾거나, 그러한 조합이 없음을 찾는 문제이다.
- 2-SAT는 불리언 식이 CNF(Conjunctive Normal Form)이고, 각 절(clause)이 2개의 리터럴(literal)로 이루어진 경우이다.
  - $(x_2 ∨ ￢x_1) ∧ (￢x_1 ∨ ￢x_2) ∧ (x_1 ∨ x_3) ∧ (￢x_2 ∨ ￢x_3) ∧ (x_1 ∨ x_4)$
  - $￢$: not, $∨$: or, $∧$: and
  - 괄호안에 있는 식을 절(clause)이라고 한다.
  - 절 안에 있는 항을 리터럴(literal)이라고 한다. 2-SAT문제는 리터럴이 2개인 절로 이루어진 식이다.
  - 이렇게 AND연산으로만 이루어진 식을 CNF라고 한다.
- 1-SAT은 $O(N)$, 2-SAT은 $O(|V| + |E|)$, 3-SAT은 NP-완전 문제임이 증명되어 다항시간 내에 풀 수 없다.

### 답의 존재성 확인 (2-SAT 3)
- 논리식을 조건문(명제)으로 바꾼다.
  - 각 절은 ∨로만 이루어져 있으므로, 다음과 같이 바꾼다고 보면 된다.
  - $(A ∨ B) = (￢A → B), (￢B → A)$
  - A가 거짓이면 B는 참이여야 하며, B가 거짓이면 A는 참이여야 한다.
- 모든 절의 명제를 그래프로 만든다.
  - $→$를 간선이라고 생각하고 만들면 된다.
- SCC로 묶은 그룹 전체가 참이거나, 전체가 거짓이여야 한다.
  - 즉 A가 SCC 그룹에 속한다고 했을 때, A와 ￢A가 동시에 존재해선 안된다.
  - $p → q$에서 $p$가 참이면 $q$는 무조건 참이여야 하지만
  - $p$가 거짓이면 $q$는 참이나 거짓이여도 된다.

### 해 구성하기 (2-SAT 4)
- $p → q$에서 $p$가 거짓이면 $q$는 참이나 거짓이여도 된다는 아이디어에서 출발한다.


In [ ]:
def tarjan(G):
  SCC, S, P = [], [], []
  D = [0] * len(G)
 
  s = [*range(len(G))]
  while s:
    x = s.pop()
    if x < 0:
      d = D[~x] - 1
      if P[-1] > d:
        SCC.append(S[d:])
        del S[d:], P[-1]
        for x in SCC[-1]:
          D[x] = -1
    elif D[x] > 0:
      while P[-1] > D[x]:
        P.pop()
    elif D[x] == 0:
      S.append(x)
      P.append(len(S))
      D[x] = len(S)
      s.append(~x)
      s += G[x]
  return SCC[::-1]
 
class TwoSat:
  def __init__(self, n):
    self.n = n
    self.graph = [[] for _ in range(2 * n)]
 
  def _imply(self, x, y):
    self.graph[x].append(y if y >= 0 else 2 * self.n + y)
 
  def either(self, x, y):
    """either x or y must be True"""
    self._imply(~x, y)
    self._imply(~y, x)
 
  def set(self, x):
    """x must be True"""
    self._imply(~x, x)
 
  def solve(self):
    SCC = tarjan(self.graph)
    order = [0] * (2 * self.n)
    for i, comp in enumerate(SCC):
      for x in comp:
        order[x] = i
    for i, v in enumerate(order[:self.n]):
      if v == order[~i]:
        return False, None
    return True, [+(v > order[~i]) for i, v in enumerate(order[:self.n])]

### 입력
- 정점 A에 대해 A의 not형 변수는 ~A로 표현한다`(-N-1)` 이 점에 주의. 
  - 즉, A번 정점의 not형을 -A라고 표현했다면, 양수에 대해서 -1을 해줘야 한다.